In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import netCDF4
import numpy as np
import os
import xarray as xr

import sys
sys.path.insert(0, '..')

from src.stream import get_stream_velocity

In [3]:
# load the NetCDF file (NetCDF4 library needed)
path_dataset = '../data/cmems_20200916_3.nc'
assert os.path.isfile(path_dataset)
cmems = xr.open_dataset(path_dataset)

In [4]:
cmems

<xarray.Dataset>
Dimensions:    (latitude: 1441, longitude: 4320, time: 3)
Coordinates:
    depth      float32 ...
  * latitude   (latitude) float32 -60.0 -59.916668 -59.833332 ... 59.916668 60.0
  * longitude  (longitude) float32 -180.0 -179.91667 ... 179.83333 179.91667
  * time       (time) datetime64[ns] 2020-09-16T12:00:00 ... 2020-09-18T12:00:00
Data variables:
    uo         (time, latitude, longitude) float32 ...
    vo         (time, latitude, longitude) float32 ...
Attributes:
    title:                              daily mean fields from Global Ocean P...
    easting:                            longitude
    northing:                           latitude
    history:                            2020/09/16 01:29:42 MERCATOR OCEAN Ne...
    source:                             MERCATOR PSY4QV3R1
    institution:                        MERCATOR OCEAN
    references:                         http://www.mercator-ocean.fr
    comment:                            CMEMS product
    Conventions:                        CF-1.4
    domain_name:                        GL12
    FROM_ORIGINAL_FILE__field_type:     mean
    field_date:                         2020-09-25 00:00:00
    field_julian_date:                  25835.0
    julian_day_unit:                    days since 1950-01-01 00:00:00
    forecast_range:                     9-day_forecast
    forecast_type:                      forecast
    bulletin_date:                      2020-09-16 00:00:00
    bulletin_type:                      operational
    FROM_ORIGINAL_FILE__longitude_min:  -180.0
    FROM_ORIGINAL_FILE__longitude_max:  179.91667
    FROM_ORIGINAL_FILE__latitude_min:   -80.0
    FROM_ORIGINAL_FILE__latitude_max:   90.0
    z_min:                              0.494025
    z_max:                              5727.917
    _CoordSysBuilder:                   ucar.nc2.dataset.conv.CF1Convention

In [5]:
# index the variable u, time 2020-09-16 and lon-lat (-10, 10) x (-20, 20)
u = cmems['uo'].sel(time='2020-09-16', longitude=slice(-10, 10), latitude=slice(-20, 20))
# extract the numpy array
u = u.data

(u < 0).sum()

52650

In [6]:
# index the variable v, time 2020-09-16 and lon-lat (-10, 10) x (-20, 20)
v = cmems['vo'].sel(time='2020-09-16', longitude=slice(-10, 10), latitude=slice(-20, 20))
# extract the numpy array
v = v.data

(v < 0).sum()

33164

In [7]:
np.concatenate((u, v)).shape

(2, 481, 241)

In [8]:
cmems.sel(time='2020-09-16', longitude=slice(-10, 10), latitude=slice(-20, 20))['latitude']

<xarray.DataArray 'latitude' (latitude: 481)>
array([-20.      , -19.916666, -19.833334, ...,  19.833334,  19.916666,
        20.      ], dtype=float32)
Coordinates:
    depth     float32 0.494025
  * latitude  (latitude) float32 -20.0 -19.916666 -19.833334 ... 19.916666 20.0
Attributes:
    valid_min:            -60.0
    valid_max:            60.0
    step:                 0.08333588
    units:                degrees_north
    unit_long:            Degrees North
    long_name:            Latitude
    standard_name:        latitude
    axis:                 Y
    _CoordinateAxisType:  Lat

# Test functions

In [9]:
lat = 40
lon = -180
date = '2020-09-16'

vel, lat, lon = get_stream_velocity(cmems, lat, lon, date)
print(vel)
print('-------\n', lat)
print('-------\n', lon)

[[[ 0.03112888  0.01831111  0.0061037  ...  0.07202368  0.06714072
    0.06469924]
  [-0.00793481 -0.01831111 -0.02746666 ...  0.06714072  0.0634785
    0.06225776]
  [-0.03662221 -0.04516739 -0.05188147 ...  0.06164739  0.06103702
    0.06286813]
  ...
  [-0.03112888 -0.02563555 -0.01831111 ...  0.08728294  0.09094515
    0.09277627]
  [-0.05981628 -0.05615406 -0.04882962 ...  0.09521775  0.09949034
    0.10254219]
  [-0.06897183 -0.06897183 -0.06408887 ...  0.10010071  0.10437331
    0.10803552]]

 [[ 0.11719108  0.15869625  0.19348735 ...  0.08362071  0.09338664
    0.09399701]
  [ 0.1312296   0.17090365  0.20569475 ...  0.08850367  0.09582812
    0.09460738]
  [ 0.14465773  0.18250069  0.21546067 ...  0.09094515  0.09582812
    0.09277627]
  ...
  [-0.1208533  -0.10315256 -0.08178961 ... -0.01098666 -0.02441481
   -0.03112888]
  [-0.126957   -0.11535996 -0.09582812 ... -0.02075259 -0.03173925
   -0.03601184]
  [-0.126957   -0.126957   -0.11597034 ... -0.02807703 -0.03723258
   -0.0